<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/tabular_neural_net_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Model For Waste Data Classification


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.
df - data_frame

In [3]:
# mount to google drive and load files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# check file is loaded OK
path = '/content/drive/My Drive/data/'
print(path)

/content/drive/My Drive/data/


In [5]:
# load train data frame
df = pd.read_csv(path + '/minimizedData.csv')



# load test data frame with material field empty
# test_df = pd.read_csv(path + '/testNoMaterialData.csv')

df.head()

,reason,origin,color,state,size,consistency,otherCode,material4,material3,material2,material,mType,composite2,composite1,cType,indirectProduct,directProduct,pType,mixedOrPure,cleanOrDirty,euralDescription,euralCode,description,/0
0,NaN,NaN,NaN,NaN,NaN,slurry,NaN,NaN,NaN,oil,organic material,Soy,NaN,dry,NaN,dry,NaN,NaN,1.0,NaN,material unsuitable for consumption or processing,20304,Soyadroes technically,NaN
1,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,wood,branches,NaN,NaN,NaN,wood,NaN,NaN,0.0,NaN,waste from forestry,20107,Branches,NaN
2,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,cocoa,caps,NaN,NaN,NaN,cocoa,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,Cocoa shells,NaN
3,NaN,NaN,NaN,debris,NaN,vast,NaN,NaN,NaN,NaN,sand,NaN,NaN,debris,NaN,debris,NaN,NaN,1.0,1.0,"mixtures of concrete, stones, tiles or ceramic...",170107,Debris with Sand,NaN
4,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,organic material,Soy,NaN,NaN,NaN,organic material,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,soya,NaN


#cats and conts 

**categorical variables - cat_names:**

what you're using to make predictions with.  (description, euralDescription, composite1, mType).
for categorical variables - will use Embeddings 

**continuos variables - cont_names:**

any catgeory that has limited number of possible (integer) choises: true/false ; fields such as cleanOrContaminated or mixedOrPure
continuos varibale names - they are sent to Neural net as pixels 

**uses processor - procs:**

a number of processes that deal with data in some ways: FillMissing, Categorify, Normalize both train and test dtata sets

In [0]:
# dependable variable field:
dep_var = 'material'
 
# can add composite1, mType
# cat_names = ['euralDescription', 'description']
cat_names = ['euralDescription', 'description',  'composite1', 'mType', 'composite2', 'consistency', 'state', 'size']
cont_names = ['mixedOrPure', 'cleanOrDirty']
# processors:
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df.iloc[950:1200].copy(), path=path + '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names)

Next:
data block api
add labels - tell which cols - column it is 

and get back data bunch
split train and test data ranges 800,1000


In [42]:
data = (TabularList.from_df(df, path=path+ '/minimizedData.csv', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(950,1200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
PCFIC, syrup, flint, solvents, zeolite...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [43]:
data.show_batch(rows=10)

euralDescription,description,composite1,mType,composite2,consistency,state,size,mixedOrPure_na,cleanOrDirty_na,mixedOrPure,cleanOrDirty,target
concrete,the railway line,concrete,#na#,#na#,vast,#na#,#na#,False,True,0.5587,0.2559,grind
#na#,c-wood,#na#,c,#na#,vast,#na#,#na#,False,True,0.5587,0.2559,wood
gypsum-based building material other than those mentioned in 17 08 01,Construction and demolition waste [plaster],#na#,#na#,#na#,vast,#na#,#na#,False,True,-1.7890,0.2559,cast
soil and stones not covered by 17 05 03,Land Bouwbakkie Pascalstraat 17,#na#,#na#,#na#,vast,#na#,#na#,False,True,0.5587,0.2559,ground
#na#,asphalt (tar-containing),asphalt,#na#,#na#,vast,debris,#na#,False,True,0.5587,0.2559,grind
plastics,ABS (regrind),#na#,#na#,#na#,vast,grinded,#na#,False,True,-1.7890,0.2559,SECTION
soil and stones not covered by 17 05 03,Soil suspected of sieving,#na#,suspected,#na#,vast,#na#,#na#,False,True,0.5587,0.2559,ground
#na#,asphalt debris containing tar,asphalt,#na#,#na#,vast,debris,#na#,False,True,0.5587,0.2559,grind
soil and stones not covered by 17 05 03,Land Overtoom 30 Zuidermeer,#na#,#na#,#na#,vast,#na#,#na#,False,True,0.5587,0.2559,ground
"mixtures of concrete, stones, tiles or ceramic products other than those mentioned in 17 01 06",MAIN OVEN SNAILS,#na#,blast furnace slag,#na#,vast,#na#,#na#,False,True,-1.7890,0.2559,snails


pass in data to tabular learner:


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [45]:
learn.fit(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.229957,1.788686,0.614754,00:00
1,1.435218,1.168200,0.762295,00:00
2,0.913820,0.974271,0.774590,00:00


In [46]:
# see into the model
# learn.model.eval()
learn.summary()

TabularModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [15]                 855        True      
______________________________________________________________________
Embedding            [104]                179,088    True      
______________________________________________________________________
Embedding            [12]                 456        True      
______________________________________________________________________
Embedding            [28]                 4,536      True      
______________________________________________________________________
Embedding            [4]                  20         True      
______________________________________________________________________
Embedding            [3]                  12         True      
______________________________________________________________________
Embedding            [14]                 686        True      
_________________________________________________

In [0]:
learn.model_dir = '/content/drive/My Drive/data/models'
learn.save('material_classifier_model3')

## Inference

data_frame (df) 

In [47]:
row = 700
print('given material -->', df.iloc[row].material)
print(df.iloc[row])

# test data
# print('test material -->', test_df.iloc[row].material)
# print(test_df.iloc[row])


given material --> grind
reason                         NaN
origin                         NaN
color                          NaN
state                    granulate
size                           NaN
consistency                   vast
otherCode                      NaN
material4                      NaN
material3                   cement
material2                     sand
material                     grind
mType                          NaN
composite2                     NaN
composite1                concrete
cType                          NaN
indirectProduct     betongranulaat
directProduct                    1
pType                          NaN
mixedOrPure                      1
cleanOrDirty                   NaN
euralDescription               NaN
euralCode                   170101
description         betongranulaat
/0                             NaN
Name: 700, dtype: object


In [57]:
# train data prediction
one_row = df.iloc[row]

# test data
# one_row = test_df.iloc[row]

predicted_category = learn.predict(one_row)[0]
total_categories = learn.predict(one_row)[1]
prediction_weights = learn.predict(one_row)[2]

print('predicted material ------> ', predicted_category)
print('')
print('total available material categories:', total_categories)
# print(total_categories)

print('prediction weights to categories:')
print(prediction_weights)


predicted material ------>  grind

total available material categories: tensor(48)
prediction weights to categories:
tensor([2.3718e-06, 1.5500e-06, 3.9022e-09, 1.2351e-06, 9.8002e-09, 4.5769e-08,
        4.7506e-08, 3.2986e-07, 1.3939e-07, 2.6613e-07, 1.0942e-07, 1.3945e-07,
        8.9792e-08, 2.7499e-08, 8.4564e-08, 4.8405e-09, 2.0181e-07, 4.4674e-07,
        1.0795e-06, 2.2936e-07, 1.2058e-07, 1.2845e-07, 1.9544e-07, 8.9341e-05,
        6.1414e-07, 5.8760e-06, 9.2457e-08, 2.4590e-06, 8.1062e-08, 1.0511e-04,
        7.7864e-07, 9.4420e-08, 1.4699e-07, 1.2710e-07, 1.8320e-06, 5.3403e-06,
        1.5306e-06, 1.0117e-08, 1.3697e-06, 1.4758e-06, 1.0466e-05, 3.6350e-08,
        7.6497e-06, 4.1584e-07, 9.4889e-08, 4.1079e-07, 4.1432e-06, 5.4043e-07,
        9.9957e-01, 1.5413e-07, 1.0330e-08, 1.4665e-06, 2.7011e-08, 5.2877e-07,
        2.0548e-06, 1.2496e-06, 1.6593e-07, 2.0961e-07, 7.8247e-07, 2.8509e-06,
        9.0731e-06, 1.7038e-06, 8.3171e-08, 2.9013e-07, 7.2693e-08, 3.3636e-07,
   


- tfm - transformations,
- train_ds - trained data set,
- df_tst = data fram etest,


In [0]:
# apply the set of transforms to your new data:
for tfm in data.tfms: df = tfm(df, is_test=True)

# Then you have to transforms the categorical variables into codes:
cats = np.stack([c.cat.codes.values for n,c in df[cat_names].items()], 1) + 1

# and normalize the continuous variables
conts = np.stack([c.astype('float32').values for n,c in df[cont_names].items()], 1)
means, stds = data.stats 
conts = (conts - means[None]) / (stds[None]+1e-7)

# And finally, you can run the model on [tensor(cats),tensor(conts)] (or some batches of it if you have a lot things to predict).